# Find duplicate questions on StackOverflow by their embeddings

In this assignment you will learn how to calculate a similarity for pieces of text. Using this approach you will know how to find duplicate questions from [StackOverflow](https://stackoverflow.com).

### Libraries

In this task you will you will need the following libraries:
- [StarSpace](https://github.com/facebookresearch/StarSpace) — a general-purpose model for efficient learning of entity embeddings from Facebook
- [Gensim](https://radimrehurek.com/gensim/) — a tool for solving various NLP-related tasks (topic modeling, text representation, ...)
- [Numpy](http://www.numpy.org) — a package for scientific computing.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — a tool for data mining and data analysis.
- [Nltk](http://www.nltk.org) — a platform to work with human language data.

### Data

The following cell will download all data required for this assignment into the folder `week3/data`.

In [1]:
import sys
sys.path.append("..")
from common.download_utils import download_week3_resources

# download_week3_resources()

### Grading
We will create a grader instace below and use it to collect your answers. Note that these outputs will be stored locally inside grader and will be uploaded to platform only after running submiting function in the last part of this assignment. If you want to make partial submission, you can run that cell any time you want.

In [2]:
from grader import Grader

In [3]:
grader = Grader()

## Word embedding

To solve the problem, you will use two different models of embeddings:

 - [Pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google which were trained on a part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases. `GoogleNews-vectors-negative300.bin.gz` will be downloaded in `download_week3_resources()`.
 - Representations using StarSpace on StackOverflow data sample. You will need to train them from scratch.

It's always easier to start with pre-trained embeddings. Unpack the pre-trained Goggle's vectors and upload them using the function [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) from gensim library with the parameter *binary=True*. If the size of the embeddings is larger than the avaliable memory, you could load only a part of the embeddings by defining the parameter *limit* (recommended: 500000).

In [4]:
import gensim

In [5]:
wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True,
                                                                limit=500000)

### How to work with Google's word2vec embeddings?

Once you have loaded the representations, make sure you can access them. First, you can check if the loaded embeddings contain a word:
    
    'word' in wv_embeddings
    
Second, to get the corresponding embedding you can use the square brackets:

    wv_embeddings['word']
 
### Checking that the embeddings are correct 
 
To prevent any errors during the first stage, we can check that the loaded embeddings are correct. You can call the function *check_embeddings*, implemented below, which runs 3 tests:
1. Find the most similar word for provided "positive" and "negative" words.
2. Find which word from the given list doesn’t go with the others.
3. Find the most similar word for the provided one.

In the right case the function will return the string *These embeddings look good*. Othervise, you need to validate the previous steps.

In [9]:
def check_embeddings(embeddings):
    error_text = "Something wrong with your embeddings ('%s test isn't correct)."
    most_similar = embeddings.most_similar(positive=['woman', 'king'], negative=['man'])
    if len(most_similar) < 1 or most_similar[0][0] != 'queen':
        return error_text % "Most similar"

    doesnt_match = embeddings.doesnt_match(['breakfast', 'cereal', 'dinner', 'lunch'])
    if doesnt_match != 'cereal':
        return error_text % "Doesn't match"
    
    most_similar_to_given = embeddings.most_similar_to_given('music', ['water', 'sound', 'backpack', 'mouse'])
    if most_similar_to_given != 'sound':
        return error_text % "Most similar to given"
    
    return "These embeddings look good."

In [10]:
print(check_embeddings(wv_embeddings))

These embeddings look good.


## From word to text embeddings

**Task 1 (Question2Vec).** Usually, we have word-based embeddings, but for the task we need to create a representation for the whole question. It could be done in different ways. In our case we will use a **mean** of all word vectors in the question. Now you need to implement the function *question_to_vec*, which calculates the question representation described above. This function should work with the input text as is without any preprocessing.

Note that there could be words without the corresponding embeddings. In this case, you can just skip these words and don't take them into account during calculating the result. If the question doesn't contain any known word with embedding, the function should return a zero vector.

In [11]:
import numpy as np

In [12]:
def question_to_vec(question, embeddings, dim=300):
    """
        question: a string
        embeddings: dict where the key is a word and a value is its' embedding
        dim: size of the representation

        result: vector representation for the question
    """
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    embedding = np.zeros(dim, dtype=np.float32)
    found_words = 0
    for word in question.split():
        if word in embeddings:
            embedding += embeddings[word]
            found_words += 1
    return embedding / max(found_words, 1)

To check the basic correctness of your implementation, run the function *question_to_vec_tests*.

In [13]:
def question_to_vec_tests():
    if (np.zeros(300) != question_to_vec('', wv_embeddings)).any():
        return "You need to return zero vector for empty question."
    if (np.zeros(300) != question_to_vec('thereisnosuchword', wv_embeddings)).any():
        return "You need to return zero vector for the question, which consists only unknown words."
    if (wv_embeddings['word'] != question_to_vec('word', wv_embeddings)).any():
        return "You need to check the corectness of your function."
    if ((wv_embeddings['I'] + wv_embeddings['am']) / 2 != question_to_vec('I am', wv_embeddings)).any():
        return "Your function should calculate a mean of word vectors."
    if (wv_embeddings['word'] != question_to_vec('thereisnosuchword word', wv_embeddings)).any():
        return "You should not consider words which embeddings are unknown."
    return "Basic tests are passed."

In [14]:
print(question_to_vec_tests())

Basic tests are passed.


You can submit embeddings for the questions from the file *test_embeddings.tsv* to earn the points. In this task you don't need to transform the text of a question somehow.

In [15]:
import nltk
nltk.download('stopwords')
from util import array_to_string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
question2vec_result = []
for question in open('data/test_embeddings.tsv'):
    question = question.strip()
    answer = question_to_vec(question, wv_embeddings)
    question2vec_result = np.append(question2vec_result, answer)

grader.submit_tag('Question2Vec', array_to_string(question2vec_result))

Current answer for task Question2Vec is: 0.019293891266
-0.0287272129208
0.0460561104119
0.0852593332529
0.0243055559695
-0.0729031041265
0.0...


Now we have a method to create a representation of any sentence and we are ready for the first evaluation. So, let's check how well our solution (Google's vectors + *question_to_vec*) will work.

## Evaluation of text similarity

We can imagine that if we use good embeddings, the cosine similarity between the duplicate sentences should be less than for the random ones. Overall, for each pair of duplicate sentences we can generate *R* random negative examples and find out the position of the correct duplicate.  

For example, we have the question *"Exceptions What really happens"* and we are sure that another question *"How does the catch keyword determine the type of exception that was thrown"* is a duplicate. But our model doesn't know it and tries to find out the best option also among questions like *"How Can I Make These Links Rotate in PHP"*, *"NSLog array description not memory address"* and *"PECL_HTTP not recognised php ubuntu"*. The goal of the model is to rank all these 4 questions (1 *positive* and *R* = 3 *negative*) in the way that the correct one is in the first place.

However, it is unnatural to count on that the best candidate will be always in the first place. So let us consider the place of the best candidate in the sorted list of candidates and formulate a metric based on it. We can fix some *K* — a reasonalble number of top-ranked elements and *N* — a number of queries (size of the sample).

### Hits@K

The first simple metric will be a number of correct hits for some *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)]$$

where $q_i$ is the i-th query, $dup_i$ is its duplicate, $topK(q_i)$ is the top K elements of the ranked sentences provided by our model and the operation $[dup_i \in topK(q_i)]$ equals 1 if the condition is true and 0 otherwise (more details about this operation could be found [here](https://en.wikipedia.org/wiki/Iverson_bracket)).


### DCG@K
The second one is a simplified [DCG metric](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K] $$

where $rank_{dup_i}$ is a position of the duplicate in the sorted list of the nearest sentences for the query $q_i$. According to this metric, the model gets a higher reward for a higher position of the correct answer. If the answer does not appear in topK at all, the reward is zero. 

### Evaluation examples

Let's calculate the described metrics for the toy example introduced above. In this case $N$ = 1 and the correct candidate for $q_1$ is *"How does the catch keyword determine the type of exception that was thrown"*. Consider the following ranking of the candidates:
1. *"How Can I Make These Links Rotate in PHP"*
2. *"How does the catch keyword determine the type of exception that was thrown"*
3. *"NSLog array description not memory address"*
4. *"PECL_HTTP not recognised php ubuntu"*

Using the ranking above, calculate *Hits@K* metric for *K = 1, 2, 4*: 
 
- [K = 1] $\text{Hits@1} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top1(q_i)] = [dup_1 \in top1(q_1)] = 0$ because the correct answer doesn't appear in the *top1* list.
- [K = 2] $\text{Hits@2} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top2(q_i)] = [dup_1 \in top2(q_1)] = 1$ because $rank_{dup_1} = 2$.
- [K = 4] $\text{Hits@4} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top4(q_i)] = [dup_1 \in top4(q_1)] = 1$

Using the ranking above, calculate *DCG@K* metric for *K = 1, 2, 4*:

- [K = 1] $\text{DCG@1} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = \frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = 0$ because the correct answer doesn't appear in the top1 list.
- [K = 2] $\text{DCG@2} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 2] = \frac{1}{\log_2{3}}$, because $rank_{dup_1} = 2$.
- [K = 4] $\text{DCG@4} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 4] = \frac{1}{\log_2{3}}$.


**Tasks 2 and 3 (HitsCount and DCGScore).** Implement the functions *hits_count* and *dcg_score* as described above. Each function has two arguments: *dup_ranks* and *k*. *dup_ranks* is a list which contains *values of ranks* of duplicates. For example, *dup_ranks* is *[2]* for the example provided above.

In [17]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
        k: number of top-ranked elements (k in Hits@k metric)

        result: return Hits@k value for current ranking
    """
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    return (np.array(dup_ranks) <= k).mean()

Test your code on the tiny examples:

In [18]:
def test_hits():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1, 1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            # from IPython.core.debugger import Tracer; Tracer()() 
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]
    
    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        # from IPython.core.debugger import Tracer; Tracer()() 
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."

    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [19]:
print(test_hits())

Basic test are passed.


In [20]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
        k: number of top-ranked elements (k in DCG@k metric)

        result: return DCG@k value for current ranking
    """
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    dup_ranks = np.array(dup_ranks)
    is_in_top_k = dup_ranks <= k
    return (is_in_top_k / np.log2(1.0 + dup_ranks)).mean()

In [21]:
def test_dcg():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]

    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, (1 + (1 / (np.log2(3)))) / 2]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."
        
    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [22]:
print(test_dcg())

Basic test are passed.


Submit results of the functions *hits_count* and *dcg_score* for the following examples to earn the points.

In [23]:
test_examples = [
    [1],
    [1, 2],
    [2, 1],
    [1, 2, 3],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [9, 5, 4, 2, 8, 10, 7, 6, 1, 3],
    [4, 3, 5, 1, 9, 10, 7, 8, 2, 6],
    [5, 1, 7, 6, 2, 3, 8, 9, 10, 4],
    [6, 3, 1, 4, 7, 2, 9, 8, 10, 5],
    [10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
]

In [24]:
hits_results = []
for example in test_examples:
    for k in range(len(example)):
        hits_results.append(hits_count(example, k + 1))
grader.submit_tag('HitsCount', array_to_string(hits_results))

Current answer for task HitsCount is: 1.0
0.5
1.0
0.5
1.0
0.333333333333
0.666666666667
1.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0
0.1
0....


In [25]:
dcg_results = []
for example in test_examples:
    for k in range(len(example)):
        dcg_results.append(dcg_score(example, k + 1))
grader.submit_tag('DCGScore', array_to_string(dcg_results))

Current answer for task DCGScore is: 1.0
0.5
0.815464876786
0.5
0.815464876786
0.333333333333
0.54364325119
0.710309917857
0.1
0.16309297...


##  First solution: pre-trained embeddings

We will work with predefined train, validation and test corpora. All the files are tab-separated, but have a different format:
 - *train* corpus contains similar sentences at the same row.
 - *validation* corpus contains the following columns: *question*, *similar question*, *negative example 1*, *negative example 2*, ... 
 - *test* corpus contains the following columns: *question*, *example 1*, *example 2*, ...

Validation corpus will be used for the intermediate validation of models. The test data will be necessary for submitting the quality of your model in the system.

Now you should upload *validation* corpus to evaluate current solution.

In [26]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [27]:
validation = read_corpus('data/validation.tsv')

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

We will use cosine distance to rank candidate questions which you need to implement in the function *rank_candidates*. The function should return a sorted list of pairs *(initial position in candidates list, candidate)*. Index of some pair corresponds to its rank (the first is the best). For example, if the list of candidates was *[a, b, c]* and the most similar is *c*, then *a* and *b*, the function should return a list *[(2, c), (0, a), (1, b)]*.

Pay attention, if you use the function *cosine_similarity* from *sklearn.metrics.pairwise* to calculate similarity because it works in a different way: most similar objects has greatest similarity. It's preferable to use a vectorized version of *cosine_similarity* function. Try to compute similarity at once and not use list comprehension. It should speed up your computations significantly.

In [44]:
from sklearn.metrics.pairwise import cosine_similarity
def rank_candidates(question, candidates, embeddings, dim=300):
    """
        question: a string
        candidates: a list of strings (candidates) which we want to rank
        embeddings: some embeddings
        dim: dimension of the current embeddings
        
        result: a list of pairs (initial position in the list, question)
    """
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    question_embedding = question_to_vec(question, embeddings, dim=dim)
    similarity = cosine_similarity(question_embedding.reshape(1,-1), [question_to_vec(c, embeddings) for c in candidates]).reshape(-1)
    return [(idx, candidates[idx]) for idx in np.argsort(similarity)[::-1]]

Test your code on the tiny examples:

In [30]:
def test_rank_candidates():
    questions = ['converting string to list', 'Sending array via Ajax fails']
    candidates = [['Convert Google results object (pure js) to Python object', 
                   'C# create cookie from string and send it',
                   'How to use jQuery AJAX for an outside domain?'], 
                  ['Getting all list items of an unordered list in PHP', 
                   'WPF- How to update the changes in list item of a list', 
                   'select2 not displaying search results']]
    results = [[(1, 'C# create cookie from string and send it'), 
                (0, 'Convert Google results object (pure js) to Python object'), 
                (2, 'How to use jQuery AJAX for an outside domain?')],
               [(0, 'Getting all list items of an unordered list in PHP'), 
                (2, 'select2 not displaying search results'), 
                (1, 'WPF- How to update the changes in list item of a list')]]
    for question, q_candidates, result in zip(questions, candidates, results):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, 300)
        if not np.all(ranks == result):
            return "Check the function."
    return "Basic tests are passed."

In [31]:
print(test_rank_candidates())

Basic tests are passed.


Now we can test the quality of the current approach. Run the next two cells to get the results. Pay attention that calculation of similarity between vectors takes time and this calculation is computed approximately in 10 minutes.

In [32]:
wv_ranking = []
for line in validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [33]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.209 | Hits@   1: 0.209
DCG@   5: 0.263 | Hits@   5: 0.311
DCG@  10: 0.279 | Hits@  10: 0.360
DCG@ 100: 0.316 | Hits@ 100: 0.548
DCG@ 500: 0.349 | Hits@ 500: 0.807
DCG@1000: 0.369 | Hits@1000: 1.000


If you did all the steps correctly, you should be frustrated by the received results. Let's try to understand why the quality is so low. First of all, when you work with some data it is necessary to have an idea how the data looks like. Print several questions from the data:

In [34]:
for line in validation[:3]:
    q, *examples = line
    print(q, *examples[:3])

How to print a binary heap tree without recursion? How do you best convert a recursive function to an iterative one? How can i use ng-model with directive in angular js flash: drawing and erasing
How to start PhoneStateListener programmatically? PhoneStateListener and service Java cast object[] to model WCF and What does this mean?
jQuery: Show a div2 when mousenter over div1 is over when hover on div1 depenting on if it is on div2 or not it should act differently How to run selenium in google app engine/cloud? Python Comparing two lists of strings for similarities


As you can see, we deal with the raw data. It means that we have many punctuation marks, special characters and unlowercased letters. In our case, it could lead to the situation where we can't find some embeddings, e.g. for the word "grid?". 

To solve this problem you should use the functions *text_prepare* from the previous assignments to prepare the data.

In [35]:
from util import text_prepare

Now transform all the questions from the validation set:

In [36]:
text_prepare('Aasd')

'aasd'

In [37]:
prepared_validation = []
for line in validation:
    prepared_validation.append([text_prepare(text) for text in line])

Let's evaluate the approach again after the preparation:

In [38]:
wv_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [39]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_prepared_ranking, k), 
                                              k, hits_count(wv_prepared_ranking, k)))

DCG@   1: 0.305 | Hits@   1: 0.305
DCG@   5: 0.375 | Hits@   5: 0.438
DCG@  10: 0.392 | Hits@  10: 0.489
DCG@ 100: 0.425 | Hits@ 100: 0.656
DCG@ 500: 0.447 | Hits@ 500: 0.830
DCG@1000: 0.465 | Hits@1000: 1.000


Now, prepare also train and test data, because you will need it in the future:

In [ ]:
def prepare_file(in_, out_):
    out = open(out_, 'w')
    for line in open(in_, encoding='utf8'):
        line = line.strip().split('\t')
        new_line = [text_prepare(q) for q in line]
        print(*new_line, sep='\t', file=out)
    out.close()

In [78]:
f = open('data/test.tsv')

In [79]:
next(f)

'How to access a structure member without knowing the name?\tGWT Servlet - ClassNotFoundException\tSlowCheetah not transforming for EntityFramework Migrations\tDevelopment tools permission I did not set\tGo to scene without using button in Flash ASC3\tobjective-c get last 2 characters of a string?\tInserting a rails link into a Google Maps infowindow\tWhen and where do I require files in a rails application?\tdo-while loop in R\tXerces-C++ DOM node line/column number location\tPython, NetCDF4 and HDF5\tGenerating a MD5 Hash with Qt\txsl scenario when a node is not present in every for each loop\tAbsolute beginners guide to working with audio in C/C++?\tNo description or item name on PayPal Payment Received email\tLaTeX annotation in MATLAB\tReplace slashes between double quote characters using sed\tiostream library not included in codeblocks\tSafe Process.Start implementation for untrusted URL strings\tTrouble linking php to database\tGoogle Analytics Api Error\tpossible to return only

In [ ]:
prepare_file('data/test.tsv', 'data/test_prepared.tsv')

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################
prepare_file('data/train.tsv', 'data/train_prepared.tsv')

**Task 4 (W2VTokenizedRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates to earn the points. The calculations should take about 3-5 minutes. Pay attention that the function *rank_candidates* returns a ranking, while in this case you should find a position in this ranking. Ranks should start with 1.

In [40]:
from util import matrix_to_string

In [41]:
w2v_ranks_results = []
prepared_test_data = 'data/test_prepared.tsv'
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, wv_embeddings, 300)
    ranked_candidates = [r[0] for r in ranks]
    w2v_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('W2VTokenizedRanks', matrix_to_string(w2v_ranks_results))

Current answer for task W2VTokenizedRanks is: 95	94	7	9	64	36	31	93	23	100	99	20	60	6	97	48	70	37	41	96	29	56	2	65	68	44	27	25	57	62	11	87	50	66	7...


## Advanced solution: StarSpace embeddings

Now you are ready to train your own word embeddings! In particular, you need to train embeddings specially for our task of duplicates detection. Unfortunately, StarSpace cannot be run on Windows and we recommend to use provided
[docker container](https://github.com/hse-aml/natural-language-processing/blob/master/Docker-tutorial.md) or other alternatives. Don't delete results of this task because you will need it in the final project.

### How it works and what's the main difference with word2vec?
The main point in this section is that StarSpace can be trained specifically for some tasks. In contrast to word2vec model, which tries to train similar embeddings for words in similar contexts, StarSpace uses embeddings for the whole sentence (just as a sum of embeddings of words and phrases). Despite the fact that in both cases we get word embeddings as a result of the training, StarSpace embeddings are trained using some supervised data, e.g. a set of similar sentence pairs, and thus they can better suit the task.

In our case, StarSpace should use two types of sentence pairs for training: "positive" and "negative". "Positive" examples are extracted from the train sample (duplicates, high similarity) and the "negative" examples are generated randomly (low similarity assumed). 

### How to choose the best params for the model?
Normally, you would start with some default choice and then run extensive experiments to compare different strategies. However, we have some recommendations ready for you to save your time:
- Be careful with choosing the suitable training mode. In this task we want to explore texts similarity which corresponds to *trainMode = 3*.
- Use adagrad optimization (parameter *adagrad = true*).
- Set the length of phrase equal to 1 (parameter *ngrams*), because we need embeddings only for words.
- Don't use a large number of *epochs* (we think that 5 should be enough).
- Try dimension *dim* equal to 100.
- To compare embeddings usually *cosine* *similarity* is used.
- Set *minCount* greater than 1 (for example, 2) if you don't want to get embeddings for extremely rare words.
- Parameter *verbose = true* could show you the progress of the training process.
- Set parameter *fileFormat* equals *labelDoc*.
- Parameter *negSearchLimit* is responsible for a number of negative examples which is used during the training. We think that 10 will be enought for this task.
- To increase a speed of training we recommend to set *learning rate* to 0.05.

Train StarSpace embeddings for unigrams on the train dataset. You don't need to change the format of the input data. Just don't forget to use prepared version of the training data. 

If you follow the instruction, the training process will take about 1 hour. The size of the embeddings' dictionary should be approximately 100 000 (number of lines in the result file). If you got significantly more than this number, try to check all the instructions above.

In [20]:
######### TRAINING HAPPENING HERE #############
!starspace train -trainFile data/train_prepared.tsv -trainMode 3 -adagrad true -ngrams 1 -epoch 5  \
 -dim 100 -similarity cosine -model week3model -minCount 2 -verbose true -fileFormat labelDoc \
 -negSearchLimit 10 -lr 0.05

Arguments: 
lr: 0.05
dim: 100
epoch: 5
maxTrainTime: 8640000
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 10
thread: 10
minCount: 2
minCountLabel: 1
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 3
fileFormat: labelDoc
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
Start to initialize starspace model.
Build dict from input file : data/train_prepared.tsv
Read 12M words
Number of words in dictionary:  110503
Number of labels in dictionary: 0
Loading data from file : data/train_prepared.tsv
Total number of examples loaded : 999624
Initialized model weights. Model size :
matrix : 110503 100
Training epoch 0: 0.05 0.01
Epoch: 100.0%  lr: 0.040000  loss: 0.009785  eta: 0h6m  tot: 0h1m33s  (20.0%)%  lr: 0.049970  loss: 0.060755  eta: 0h13m  tot: 0h0m0s  (0.1%)s  (0.4%)2.9%  lr: 0.049780  loss: 0.037071  eta: 0h8m  tot: 0h0m3s  (0.6%)8.7%  lr: 0.049129  loss: 0.025551  eta: 0h7m  tot: 0h0m8s  (1.7%)9.5%  lr: 0.049059  loss: 0.0

Epoch: 100.0%  lr: 0.020000  loss: 0.001816  eta: 0h2m  tot: 0h4m30s  (60.0%)%  lr: 0.029930  loss: 0.001981  eta: 0h4m  tot: 0h3m2s  (40.1%)4.8%  lr: 0.029439  loss: 0.001949  eta: 0h4m  tot: 0h3m5s  (41.0%)6.1%  lr: 0.029389  loss: 0.001886  eta: 0h4m  tot: 0h3m6s  (41.2%)7.4%  lr: 0.029209  loss: 0.001935  eta: 0h4m  tot: 0h3m8s  (41.5%)10.0%  lr: 0.029019  loss: 0.001930  eta: 0h4m  tot: 0h3m10s  (42.0%)%)23.5%  lr: 0.027488  loss: 0.001836  eta: 0h4m  tot: 0h3m22s  (44.7%)24.6%  lr: 0.027367  loss: 0.001830  eta: 0h4m  tot: 0h3m23s  (44.9%)26.5%  lr: 0.027177  loss: 0.001825  eta: 0h4m  tot: 0h3m25s  (45.3%)33.6%  lr: 0.026346  loss: 0.001835  eta: 0h4m  tot: 0h3m32s  (46.7%)35.8%  lr: 0.026206  loss: 0.001833  eta: 0h3m  tot: 0h3m34s  (47.2%)39.3%  lr: 0.025876  loss: 0.001832  eta: 0h3m  tot: 0h3m37s  (47.9%)39.6%  lr: 0.025846  loss: 0.001833  eta: 0h3m  tot: 0h3m37s  (47.9%)43.0%  lr: 0.025546  loss: 0.001832  eta: 0h3m  tot: 0h3m40s  (48.6%)46.8%  lr: 0.025235  loss: 0.001816

And now we can compare the new embeddings with the previous ones. You can find trained word vectors in the file *[model_file_name].tsv*. Upload the embeddings from StarSpace into a dict. 

In [71]:
starspace_embeddings = dict()
for line in open('week3model.tsv', encoding='utf-8'):
    q, *ex = line.strip().split('\t')
    if len(ex) < 100:
        print(line)
        break
    starspace_embeddings[q] = np.array(ex, dtype=np.float32)

java



In [75]:
f = open('data/train_prepared.tsv', encoding='utf-8')

In [77]:
l = next(f)
l

'html 5 canvas javascript use making interactive drawing tool\tevent handling geometries threejs\n'

In [64]:
l = next(f)
l

'file\t0.0208124\t0.0344047\t-0.0344049\t0.0657013\t0.020469\t-0.0194569\t-0.0400735\t0.0586822\t0.0689199\t-0.026807\t0.0304413\t-0.0525206\t-0.066155\t-0.00605997\t0.0316914\t-0.0107329\t0.0349454\t-0.0328102\t0.0136964\t0.0315276\t-0.0181975\t0.00870207\t0.0285195\t-0.0397131\t-0.0215149\t0.0256014\t-0.0295706\t0.0409087\t0.0398968\t-0.0052346\t-0.00922728\t-0.0258903\t-0.00345767\t-0.0104773\t0.0423385\t0.0317324\t0.0153624\t-0.0249137\t-0.00766188\t0.0154943\t-0.0506855\t-0.0440863\t0.00276399\t0.0335753\t0.0321486\t-0.036046\t-0.0128471\t0.000851746\t0.103289\t0.0815258\t-0.016628\t0.00627557\t0.0394692\t-0.00855764\t0.0251038\t0.0451494\t-0.10446\t-0.00516351\t0.0206225\t0.038589\t-0.00078419\t-0.0135166\t-0.039732\t-0.0781202\t-0.0166548\t0.0201908\t-0.013822\t-0.0227522\t-0.0191417\t0.0288483\t-0.062627\t-0.00626662\t-0.00877586\t0.0235134\t0.022744\t-0.00708179\t0.0160009\t0.033836\t0.0167684\t0.0520947\t-0.0625077\t-0.0771654\t-0.0530052\t0.00936241\t-0.0110048\t-0.0401759\t

In [65]:
l = next(f)
l

'android\t0.0330639\t-0.0195711\t0.00182071\t0.0217052\t-0.0280802\t0.0191014\t0.0480053\t0.0307314\t-0.0966976\t0.0716527\t0.00957561\t-0.0235741\t-0.0710098\t0.0300581\t-0.0273113\t0.0698121\t0.00717061\t-0.0402271\t-0.0616039\t-0.0196532\t0.032136\t0.0669243\t0.0257606\t-0.0207143\t0.00776075\t0.0356336\t0.0612106\t0.0465131\t-0.020977\t-0.0376891\t0.0266019\t-0.0491257\t-0.0778359\t0.059647\t0.009146\t-0.0670566\t-0.055453\t-0.0202815\t-0.0603593\t0.0123468\t-0.0191864\t-0.00873635\t0.0222912\t0.0288145\t-0.0061137\t0.0710339\t0.0722912\t0.113893\t0.0501821\t-0.0646072\t-0.0414661\t-0.0943278\t0.0165119\t0.0348931\t0.0473046\t-0.0202202\t0.0243119\t0.00786317\t-0.0929114\t0.0286816\t-0.0286559\t-0.0318042\t0.0245244\t0.0365115\t0.0395255\t-0.00529468\t0.0813587\t0.0186529\t0.000917076\t-0.0298803\t0.000398173\t-0.0392098\t-0.0263845\t0.0131706\t0.0537836\t0.0200519\t-0.0701383\t-0.0101738\t0.0120453\t0.0480176\t-0.0641946\t-0.00757235\t-0.049224\t0.0401026\t-0.0438248\t0.144037\t-0

In [66]:
l = next(f)
l

'get\t-0.0116774\t0.070582\t0.0158432\t0.0193376\t0.00104479\t-0.0285625\t-0.000146783\t0.0211616\t-0.0207828\t0.00693713\t0.0356723\t0.0382138\t0.00326956\t0.0465548\t-0.0141246\t0.00279946\t0.039973\t-0.0203557\t-0.00645734\t0.0366549\t0.0101151\t0.0140132\t-0.0173132\t-0.0129756\t0.0287837\t0.00062601\t0.0357466\t0.0220668\t0.0670331\t-0.000237744\t-0.0306797\t-0.0143791\t0.0121673\t-0.0503801\t-0.0233542\t0.0275014\t-0.0421322\t-0.0125225\t-0.000735657\t-0.00621687\t0.000296384\t0.0400397\t0.00707109\t0.0324031\t0.027234\t-0.0266162\t0.0617837\t0.0229032\t-0.031907\t-0.0233232\t0.0145715\t0.0363197\t-0.0344193\t0.0377439\t0.0313878\t-0.0110791\t0.00981241\t0.0135075\t-0.0133405\t0.0864139\t0.0330966\t-0.01392\t0.0054076\t-0.0175608\t-0.0301374\t-0.0275253\t0.00403108\t-0.0828083\t-0.0251905\t-0.019045\t0.00576671\t0.0259508\t-0.0323469\t0.0115187\t0.00232662\t-0.00498826\t0.0169493\t0.0160852\t-0.0125209\t0.00984904\t-0.0721773\t0.0248424\t0.0348976\t-0.0247185\t0.0244611\t0.013491

In [67]:
l = next(f)
l

'use\t0.0137033\t-0.00887483\t0.035095\t-0.0105588\t-0.00342779\t-0.00230318\t0.00344651\t-0.0142561\t-0.0146811\t-0.00927209\t0.0188775\t-0.00272003\t-0.00681423\t0.0111948\t-0.0132769\t0.0133479\t-0.0106998\t0.0431333\t0.0176371\t0.0267702\t0.00272782\t0.0218289\t0.0388948\t-0.0245845\t-0.0281702\t-0.0104557\t0.0186122\t0.00591279\t0.0474088\t-0.0103159\t0.000377793\t0.0343758\t-0.0135752\t0.0174126\t-0.0292738\t0.00164773\t0.0133836\t0.0180813\t0.0275989\t0.00668033\t-0.0185096\t0.0116124\t-0.0437978\t-0.0284374\t-0.0243909\t-0.0202719\t-0.0266302\t0.0516295\t-0.0191912\t-0.0283222\t-0.0131145\t-0.0172813\t0.0131034\t-0.00138217\t0.00603364\t0.0296496\t-0.0112317\t-0.00407906\t0.012194\t-0.0107108\t-0.0242655\t0.0234042\t0.0301197\t-0.00937211\t0.0153755\t-2.33994e-05\t-0.0218179\t-0.0737267\t0.0408734\t-0.0379083\t0.00833017\t-0.000499396\t-0.0245212\t-0.00939275\t0.0146459\t-0.0345721\t0.033891\t0.035432\t0.0220838\t-0.0139029\t-0.00342063\t-0.00898115\t-0.0184814\t0.0412357\t0.00

In [68]:
l = next(f)
l

'java\t0.0386055\t0.0201119\t0.0328729\t-0.0957672\t0.0114957\t-0.0278843\t-0.0767195\t0.0291077\t-0.0134255\t0.131172\t-0.0105205\t-0.0185705\t-0.0263503\t0.0163467\t-0.0312587\t-0.0166329\t0.0212224\t0.0568228\t-0.00271723\t-0.0320586\t0.0531453\t-0.0392828\t-0.0165459\t-0.101559\t-0.0284917\t0.0374062\t-0.0142094\t-0.0104406\t-0.0789606\t-0.0511784\t0.0373867\t-0.0766869\t0.00306198\t-0.00580975\t-0.0493982\t0.0426011\t-0.0667633\t-0.0393443\t-0.0033275\t-0.0215191\t-0.00194586\t0.0408036\t0.0276181\t0.0207256\t-0.0435252\t-0.0384805\t0.0480162\t-0.0192185\t0.0592976\t-0.00716955\t-0.0445932\t-0.0596022\t0.0773285\t0.0632403\t0.0858511\t-0.009515\t0.0503408\t-0.00910577\t-0.0754992\t-0.021798\t0.017973\t0.0357145\t0.0177712\t0.00202541\t-0.011262\t-0.0295227\t-0.0521981\t0.0465615\t-0.0203596\t-0.0225617\t-0.0901211\t-0.0617219\t-0.0669932\t-0.0520492\t-0.0156596\t-0.0327914\t-0.0794136\t0.116028\t-0.000526737\t0.0454618\t-0.0797828\t-0.0497482\t0.00378102\t0.0569748\t0.0192137\t0.0

In [70]:
l.strip().split('\t')

['java',
 '0.0386055',
 '0.0201119',
 '0.0328729',
 '-0.0957672',
 '0.0114957',
 '-0.0278843',
 '-0.0767195',
 '0.0291077',
 '-0.0134255',
 '0.131172',
 '-0.0105205',
 '-0.0185705',
 '-0.0263503',
 '0.0163467',
 '-0.0312587',
 '-0.0166329',
 '0.0212224',
 '0.0568228',
 '-0.00271723',
 '-0.0320586',
 '0.0531453',
 '-0.0392828',
 '-0.0165459',
 '-0.101559',
 '-0.0284917',
 '0.0374062',
 '-0.0142094',
 '-0.0104406',
 '-0.0789606',
 '-0.0511784',
 '0.0373867',
 '-0.0766869',
 '0.00306198',
 '-0.00580975',
 '-0.0493982',
 '0.0426011',
 '-0.0667633',
 '-0.0393443',
 '-0.0033275',
 '-0.0215191',
 '-0.00194586',
 '0.0408036',
 '0.0276181',
 '0.0207256',
 '-0.0435252',
 '-0.0384805',
 '0.0480162',
 '-0.0192185',
 '0.0592976',
 '-0.00716955',
 '-0.0445932',
 '-0.0596022',
 '0.0773285',
 '0.0632403',
 '0.0858511',
 '-0.009515',
 '0.0503408',
 '-0.00910577',
 '-0.0754992',
 '-0.021798',
 '0.017973',
 '0.0357145',
 '0.0177712',
 '0.00202541',
 '-0.011262',
 '-0.0295227',
 '-0.0521981',
 '0.0465615'

In [58]:
[len(e) for e in starspace_embeddings.values()]

[100,
 99,
 100,
 100,
 100,
 0,
 99,
 99,
 0,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 99,
 100,
 0,
 0,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 0,
 100,
 100,
 100,
 100,
 99,
 100,
 0,
 0,
 100,
 100,
 100,
 100,
 0,
 100,
 99,
 0,
 100,
 99,
 100,
 100,
 100,
 0,
 99,
 99,
 100,
 100,
 100,
 0,
 0,
 0,
 0,
 100,
 99,
 100,
 0,
 0,
 100,
 100,
 100,
 100,
 99,
 99,
 100,
 100,
 100,
 100,
 100,
 99,
 99,
 0,
 100,
 99,
 100,
 0,
 100,
 100,
 100,
 0,
 99,
 0,
 99,
 99,
 100,
 100,
 99,
 99,
 100,
 0,
 0,
 100,
 99,
 100,
 100,
 100,
 100,
 99,
 100,
 0,
 100,
 100,
 0,
 100,
 99,
 99,
 100,
 100,
 100,
 0,
 99,
 100,
 0,
 99,
 100,
 100,
 100,
 0,
 100,
 100,
 100,
 99,
 100,
 100,
 100,
 100,
 99,
 0,
 100,
 100,
 99,
 100,
 0,
 100,
 100,
 100,
 100,
 100,
 100,
 99,
 0,
 99,
 100,
 100,
 100,
 0,
 100,
 100,
 100,
 100,
 0,
 100,
 0,
 99,
 100,
 0,
 100,
 100,
 0,
 100,
 99,
 0,
 100,
 99,
 100,
 100,
 0,
 99,
 0,
 100,
 0,
 100,
 10

In [45]:
ss_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ss_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

ValueError: operands could not be broadcast together with shapes (100,) (0,) (100,) 

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking, k), 
                                               k, hits_count(ss_prepared_ranking, k)))

Due to training for the particular task with the supervised data, you should expect to obtain a higher quality than for the previous approach. In additiion, despite the fact that StarSpace's trained vectors have a smaller dimension than word2vec's, it provides better results in this task.

**Task 5 (StarSpaceRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates for trained representation.

In [ ]:
starspace_ranks_results = []
prepared_test_data = ######### YOUR CODE HERE #############
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ranked_candidates = [r[0] for r in ranks]
    starspace_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('StarSpaceRanks', matrix_to_string(starspace_ranks_results))

Please, **don't remove** the file with these embeddings because you will need them in the final project.

### Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. <b>Note:</b> Token expires 30 minutes after generation.

In [75]:
STUDENT_EMAIL = 'nomenou@gmail.com'
STUDENT_TOKEN = 'I3JldM9ST4ZtlRtT'
grader.status()

You want to submit these parts:
Task Question2Vec: 0.01929389126598835
-0.02872721292078495
0.0460561104118824
0.0852593332529068
0.0243055559694767
-0...
Task HitsCount: 1.0
0.5
1.0
0.5
1.0
0.3333333333333333
0.6666666666666666
1.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1....
Task DCGScore: 1.0
0.5
0.8154648767857288
0.5
0.8154648767857288
0.3333333333333333
0.5436432511904858
0.7103099178...
Task W2VTokenizedRanks: 95	94	7	9	64	37	32	93	24	100	98	17	60	6	97	49	70	38	42	96	30	21	2	65	67	45	27	26	57	62	11	88	56	66	7...
Task StarSpaceRanks: ----------...


If you want to submit these answers, run cell below

In [76]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
